# Face Mask Detection 

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image 

### Importing data

In [3]:
with_mask_files = os.listdir("data\with_mask")
print(with_mask_files[0:5])

['with_mask_1.jpg', 'with_mask_10.jpg', 'with_mask_100.jpg', 'with_mask_1000.jpg', 'with_mask_1001.jpg']


In [4]:
without_mask_files = os.listdir("data\without_mask")
print(without_mask_files[0:5])

['without_mask_1.jpg', 'without_mask_10.jpg', 'without_mask_100.jpg', 'without_mask_1000.jpg', 'without_mask_1001.jpg']


In [5]:
print('Number of with mask images:', len(with_mask_files))
print('Number of without mask images:', len(without_mask_files))

Number of with mask images: 3725
Number of without mask images: 3828


### Creating labels for images 

In [6]:
with_mask_labels = [1]*3725
without_mask_labels = [0]*3828

In [7]:
labels = with_mask_labels + without_mask_labels

In [8]:
with_mask_path = "data\with_mask"

data = []

for img_file in with_mask_files:

  image = Image.open(with_mask_path+ "\\"+ img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

without_mask_path = "data\without_mask"

for img_file in without_mask_files:

  image = Image.open(without_mask_path+ "\\"+ img_file)
  image = image.resize((128,128))
  image = image.convert('RGB')
  image = np.array(image)
  data.append(image)

c:\Face Mask detection\venev\lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [9]:
X = np.array(data)
Y = np.array(labels)

In [10]:
#Scaling data

In [11]:
X = X/255

In [12]:
#Splitting train data and test data 

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Constructing CNN architecture 

In [15]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))


model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())

c:\Face Mask detection\venev\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,400,641 (28.23 MB)

 Trainable params: 7,400,641 (28.23 MB)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
Y_test = Y_test.reshape(-1, 1)
model.fit(X_train, Y_train, epochs=10, verbose=1, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 23s 140ms/step - accuracy: 0.6671 - loss: 0.7138 - val_accuracy: 0.8784 - val_loss: 0.2924
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 19s 121ms/step - accuracy: 0.8765 - loss: 0.3092 - val_accuracy: 0.9049 - val_loss: 0.2418
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.8812 - loss: 0.2876 - val_accuracy: 0.9024 - val_loss: 0.2611
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - accuracy: 0.9008 - loss: 0.2202 - val_accuracy: 0.9090 - val_loss: 0.2673
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 132ms/step - accuracy: 0.9182 - loss: 0.2024 - val_accuracy: 0.9363 - val_loss: 0.1876
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 132ms/step - accuracy: 0.9289 - loss: 0.1605 - val_accuracy: 0.9347 - val_loss: 0.1988
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 0.9507 - loss: 0.1238 - val_accuracy: 0.9355 - val_loss: 0.2166
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 20s 131ms/step - accuracy: 0.9566 - loss: 0

In [17]:
y_prob=model.predict(X_test)
y_pred=np.where(y_prob>0.5,1,0)

48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step


In [18]:
y_pred

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]])

### Model Performance

In [19]:
Y_test

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]])

In [20]:
from sklearn.metrics import accuracy_score,recall_score,classification_report

In [21]:
acc=accuracy_score(Y_test,y_pred)
recall=recall_score(Y_test,y_pred)
cr=classification_report(Y_test,y_pred)
print(f"accuracy is {acc*100:.2f}%")
print(f"recall is {recall*100:.2f}%")
print(cr)

accuracy is 93.18%
recall is 93.54%
              precision    recall  f1-score   support

           0       0.94      0.93      0.93       768
           1       0.93      0.94      0.93       743

    accuracy                           0.93      1511
   macro avg       0.93      0.93      0.93      1511
weighted avg       0.93      0.93      0.93      1511



### Implementing VGG16 model 

In [36]:
image_size = (128,128)
channels = 3
image_shape = (image_size[0],image_size[1], channels)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, BatchNormalization, Dense, Dropout
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
vgg = VGG16(input_shape=image_shape, weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False
  
model = Sequential([
    vgg,
    Flatten(),
    BatchNormalization(),
    Dense(256, activation ='relu'),
    Dropout(rate=0.2),
    BatchNormalization(),
    Dense(128, activation ='relu'),
    Dropout(rate=0.2),
    BatchNormalization(),
    Dense(64, activation ='relu'),
    Dropout(rate=0.2),
    
    Dense(1, activation ='sigmoid')
    
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ ?                           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

None


In [37]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
Y_test = Y_test.reshape(-1, 1)
model.fit(X_train, Y_train, epochs=10, verbose=1, validation_split=0.2, callbacks=[early_stopping])
#model.fit(X_train, Y_train, epochs=5)

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.8758 - loss: 0.2869 - val_accuracy: 0.9702 - val_loss: 0.0829
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9670 - loss: 0.0857 - val_accuracy: 0.9727 - val_loss: 0.0817
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 243s 2s/step - accuracy: 0.9806 - loss: 0.0562 - val_accuracy: 0.9702 - val_loss: 0.0881
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.9903 - loss: 0.0315 - val_accuracy: 0.9719 - val_loss: 0.0905
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 246s 2s/step - accuracy: 0.9914 - loss: 0.0266 - val_accuracy: 0.9719 - val_loss: 0.0990
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step - accuracy: 0.9886 - loss: 0.0309 - val_accuracy: 0.9752 - val_loss: 0.1022
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.9913 - loss: 0.0268 - val_accuracy: 0.9719 - val_loss: 0.1119


In [40]:
y_prob=model.predict(X_test)
y_pred=np.where(y_prob>0.5,1,0)

48/48 ━━━━━━━━━━━━━━━━━━━━ 124s 2s/step


### Model Performance

In [43]:
from sklearn.metrics import accuracy_score,recall_score,classification_report

In [44]:
acc=accuracy_score(Y_test,y_pred)
recall=recall_score(Y_test,y_pred)
cr=classification_report(Y_test,y_pred)
print(f"accuracy is {acc*100:.2f}%")
print(f"recall is {recall*100:.2f}%")
print(cr)

accuracy is 98.48%
recall is 98.12%
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       768
           1       0.99      0.98      0.98       743

    accuracy                           0.98      1511
   macro avg       0.98      0.98      0.98      1511
weighted avg       0.98      0.98      0.98      1511



In [45]:
import cv2

In [55]:
input_image_path = "data\without_mask\without_mask_4.jpg"
input_image = cv2.imread(input_image_path)
cv2.imshow("Input Image", input_image)
image_resized = cv2.resize(input_image, (128, 128))
image_scaled = image_resized.astype('float32') / 255.0
image_reshaped = np.reshape(image_scaled, [1, 128, 128, 3])
prediction = model.predict(image_reshaped)
if prediction[0] <= 0.5:
    print('The person in the image is wearing a mask')
else:
    print('The person in the image is not wearing a mask')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
[0.96769685]
The person in the image is not wearing a mask


In [48]:
import pickle 
pickle.dump(model,open('model.pkl','wb'))